In [ ]:
print("Utviklingsmiljøet er klart!")

# Beskrivelse av værdata-innhenting med Frost og OpenCage API

## Introduksjon
Denne Python-koden henter historiske værdata (temperatur, nedbør, vindhastighet, luftfuktighet) for en valgt lokasjon i Norge.  
Det bruker API-tjenestene **Frost** (fra Meteorologisk Institutt) og **OpenCage** (for geokoding).

---

Dette er den første koden som skal kjøres, deretter kjøres datarensing, så plottingen. De er markert med 1,2 og 3.

---

## Funksjoner

- **Geokoding**:  
  Bruker OpenCage API til å finne bredde- og lengdegrader for en brukerinndata-lokasjon (f.eks. Bergen, Oslo).
  
- **Finn nærmeste værstasjoner**:  
  Henter alle værstasjoner fra Frost API og beregner avstanden fra brukerens lokasjon til hver stasjon.
  
- **Datainnhenting**:  
  For hver værparameter (element) forsøker skriptet å hente observasjoner fra de nærmeste stasjonene.

- **Databehandling**:  
  Samler dataene i en Pandas DataFrame, fjerner unødvendige kolonner, og lagrer datasett for hvert element til separate CSV-filer.

---

## Brukte bibliotek

- `requests` — for HTTP-kall til API-er
- `pandas` — for å organisere og lagre data
- `os`, `sys` — for filstier og systemmanipulering
- `datetime`, `timedelta` — for håndtering av datoer
- `opencage.geocoder` — for geokoding av lokasjon
- `geopy.distance` — for å beregne geografiske avstander
- Egendefinert modul: `API_Key_Manager_class` for håndtering av API-nøkler

---

## CSV-filer som genereres

- `temperature_data.csv`
- `precipitation_data.csv`
- `wind_speed_data.csv`
- `humidity_data.csv`

Hver fil inneholder historiske observasjoner for valgt lokasjon de siste 7 årene.

---

## Forutsetninger

- Du må ha installert nødvendige Python-biblioteker (`requests`, `pandas`, `geopy`, `opencage`).

---

# Oppsummering
Denne koden automatiserer en omfattende prosess fra **lokasjonssøk** til **datainnsamling**, **databehandling**, og **datapreparering**. Det er spesielt nyttig for prosjekter innen datavitenskap, væranalyse eller maskinlæring.

---

In [4]:
# Importer nødvendige biblioteker
import os
import sys
from datetime import datetime, timedelta
from opencage.geocoder import OpenCageGeocode

# Legg til sti for egendefinerte moduler
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

# Importer APIKeyManager og WeatherDataFetcher fra modulen
from API_Key_Manager_class import APIKeyManager
from Var_data_henting import WeatherDataFetcher

# Initialiser APIKeyManager
api_key_manager = APIKeyManager()

# Hent Frost API-nøkkelen
client_id = api_key_manager.get_api_key("Frost")

# Hent OpenCage API-nøkkelen
api_key = api_key_manager.get_api_key("Opencage")

# Sett opp OpenCageGeocode med API-nøkkelen
geolocator = OpenCageGeocode(api_key)

# Henter inn lokasjon fra brukeren med validering
while True:
    location_input = input("Enter the location (e.g., Bergen, Oslo, Trondheim, Stavanger, Tromsø): ")
    location = geolocator.geocode(location_input)
    if location:
        location_coords = (location[0]['geometry']['lat'], location[0]['geometry']['lng'])
        print(f"Coordinates for {location_input}: {location_coords}")
        break
    else:
        print("Invalid location. Please try again.")

# Initialiser WeatherDataFetcher med nødvendige parametere
observations_endpoint = 'https://frost.met.no/observations/v0.jsonld'
stations_endpoint = 'https://frost.met.no/sources/v0.jsonld'
weather_fetcher = WeatherDataFetcher(client_id, observations_endpoint, stations_endpoint)

# Hent værstasjoner
weather_stations_coords = weather_fetcher.fetch_stations()

# Regn ut avstanden til værstasjonene fra lokasjonen
from geopy.distance import geodesic
distances = {station: geodesic(location_coords, coords).km for station, coords in weather_stations_coords.items()}

# Sorter værstasjonene etter avstand til lokasjonen
sorted_stations = sorted(distances, key=distances.get)

# Hent ut dagens dato og datoen x dager tilbake
now = datetime.now()
days = 365 * 7
offset = now - timedelta(days)
current_date = now.strftime("%Y-%m-%d")
historic_date = offset.strftime("%Y-%m-%d")

# Elementene vi ønsker å hente data for
elements = [
    'mean(air_temperature P1D)', 
    'sum(precipitation_amount P1D)', 
    'mean(wind_speed P1D)',
    'mean(relative_humidity P1D)'
]

# Hent data for lokasjonen
data = weather_fetcher.fetch_data_by_element(sorted_stations, elements, historic_date, current_date)

# Prosesser dataene til en Pandas DataFrame
df = weather_fetcher.process_data(data)

# Lagrer dataene i CSV-filer (overskriver eksisterende fil)

# Filtrerer og lagrer data for temperatur
temperature_data = df[df['elementId'] == 'mean(air_temperature P1D)']
temperature_data.to_csv("../data/temperature_data.csv", index=False)

# Filtrerer og lagrer data for nedbør
precipitation_data = df[df['elementId'] == 'sum(precipitation_amount P1D)']
precipitation_data.to_csv("../data/precipitation_data.csv", index=False)

# Filtrerer og lagrer data for vindhastighet
wind_speed_data = df[df['elementId'] == 'mean(wind_speed P1D)']
wind_speed_data.to_csv("../data/wind_speed_data.csv", index=False)

# Filtrerer og lagrer data for relativ luftfuktighet
humidity_data = df[df['elementId'] == 'mean(relative_humidity P1D)']
humidity_data.to_csv("../data/humidity_data.csv", index=False)

Coordinates for Oslo: (59.9133301, 10.7389701)
Henter data for element: mean(air_temperature P1D)
Error retrieving mean(air_temperature P1D) data from station SN18317. Trying next station...
Error retrieving mean(air_temperature P1D) data from station SN18645. Trying next station...
Data retrieved for mean(air_temperature P1D) from station SN18690!
Henter data for element: sum(precipitation_amount P1D)
Data retrieved for sum(precipitation_amount P1D) from station SN18317!
Henter data for element: mean(wind_speed P1D)
Error retrieving mean(wind_speed P1D) data from station SN18317. Trying next station...
Error retrieving mean(wind_speed P1D) data from station SN18645. Trying next station...
Error retrieving mean(wind_speed P1D) data from station SN18690. Trying next station...
Error retrieving mean(wind_speed P1D) data from station SN18310. Trying next station...
Error retrieving mean(wind_speed P1D) data from station SN499999010. Trying next station...
Error retrieving mean(wind_speed 